In [2]:
import os
import math
import optuna
import wandb
from optuna.pruners import SuccessiveHalvingPruner
from optuna.exceptions import TrialPruned
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoProcessor,
    Trainer,
    TrainingArguments,
    TrainerCallback
)
from peft import get_peft_model, LoraConfig, TaskType
import torch
import numpy as np
import evaluate
from dotenv import load_dotenv
from huggingface_hub import login
from transformers import EarlyStoppingCallback

# -- 1) Setup HF login and global objects --
load_dotenv("secrets.env")
hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)

MODEL_NAME = "microsoft/Florence-2-base-ft"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = AutoProcessor.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Load full dataset
raw_ds = load_dataset("SimulaMet-HOST/Kvasir-VQA")['raw']

# Load metrics
bleu, meteor, rouge = map(evaluate.load, ["bleu", "meteor", "rouge"])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # decode predictions and labels
    preds = np.argmax(logits[0], axis=-1)
    pred_texts = processor.tokenizer.batch_decode(preds, skip_special_tokens=True)
    label_ids = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
    label_texts = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    pred_texts = [p.strip() for p in pred_texts]
    label_texts = [l.strip() for l in label_texts]
    return {"rougeL": rouge.compute(predictions=pred_texts, references=label_texts)["rougeL"]}

# Callback to report metrics to Optuna and prune
class OptunaPruningCallback(TrainerCallback):
    def __init__(self, trial, metric_name: str):
        self.trial = trial
        self.metric_name = metric_name

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        score = metrics.get(self.metric_name)
        step = state.global_step
        self.trial.report(score, step)
        if self.trial.should_prune():
            raise TrialPruned(f"Trial was pruned at step {step}")
        return control

# Objective for Optuna
def objective(trial):
    # Initialize WandB
    run_name = f"hparam_trial_{trial.number}"
    wandb.init(
        project="florence2_hparam_trial",
        name=run_name,
        reinit=True
    )

    # -- 2) Sample hyperparameters --
    lr       = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
    train_bs = trial.suggest_categorical("per_device_train_batch_size", [2, 4])
    grad_acc = trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4])
    weight_decay = trial.suggest_uniform("weight_decay", 0.0, 0.1)
    lora_r      = trial.suggest_categorical("lora_r", [4, 8, 16])
    lora_alpha  = trial.suggest_categorical("lora_alpha", [8, 16, 32])
    lora_dropout= trial.suggest_uniform("lora_dropout", 0.0, 0.3)

    # Log hyperparameters to WandB
    wandb.config.update({
        "lr": lr,
        "train_bs": train_bs,
        "grad_acc": grad_acc,
        "weight_decay": weight_decay,
        "lora_r": lora_r,
        "lora_alpha": lora_alpha,
        "lora_dropout": lora_dropout,
    })

    # -- 3) Prepare small train/val split (2.5% coverage) --
    DATASET_COVERAGE = 0.025
    SEED = 7
    TEST_SIZE = 0.1
    
    if DATASET_COVERAGE < 1:
        dataset = raw_ds.train_test_split(test_size=DATASET_COVERAGE, seed=SEED, shuffle=True)
        dataset = dataset['test'].train_test_split(test_size=TEST_SIZE, seed=SEED)
        train_dataset, val_dataset = dataset['train'], dataset['test']
    else:
        dataset = ds.train_test_split(test_size=TEST_SIZE, seed=SEED)
        train_dataset, val_dataset = dataset['train'], dataset['test']
    
    # -- 4) Load model, processor, and apply LoRA --
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=True)
    peft_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "lm_head", "fc2"],
        task_type=TaskType.CAUSAL_LM,
        lora_dropout=lora_dropout,
        bias="none",
        inference_mode=False
    )
    model = get_peft_model(model, peft_config)
    model = model.to(DEVICE)

    # -- 5) Collate function --
    def collate_fn(batch):
        questions = [x['question'] for x in batch]
        images = [x['image'].convert("RGB") if x['image'].mode != "RGB" else x['image'] for x in batch]
        answers = [x['answer'] for x in batch]

        inputs = processor(text=questions, images=images, return_tensors="pt", padding=True)
        labels = processor.tokenizer(answers, return_tensors="pt", padding=True).input_ids
        labels[labels == processor.tokenizer.pad_token_id] = -100
        inputs["labels"] = labels
        return inputs

    # -- 6) Compute steps --
    TRAIN_DATASET_SIZE = len(train_dataset)
    NUM_EPOCHS = 1
    effective_bs = train_bs * grad_acc
    steps_per_epoch = math.ceil(TRAIN_DATASET_SIZE / effective_bs)
    max_steps = steps_per_epoch * NUM_EPOCHS

    # -- 7) TrainingArguments with pruning --
    training_args = TrainingArguments(
        output_dir=run_name,
        per_device_train_batch_size=train_bs,
        gradient_accumulation_steps=grad_acc,
        per_device_eval_batch_size=train_bs,
        eval_accumulation_steps=1,
        max_steps=max_steps,
        learning_rate=lr,
        weight_decay=weight_decay,
        logging_steps=10,
        save_strategy="no",
        # save_steps=steps_per_epoch,
        eval_strategy="steps",
        eval_steps=steps_per_epoch,
        report_to=["wandb"],
        # load_best_model_at_end=True,
        metric_for_best_model="rougeL",
        greater_is_better=True,
        fp16=True,
        remove_unused_columns=False,
    )

    pruning_cb = OptunaPruningCallback(trial, metric_name="eval_rougeL")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=collate_fn,
        compute_metrics=compute_metrics,
        callbacks=[pruning_cb, EarlyStoppingCallback(early_stopping_patience=2)]
    )

    # -- 8) Train & evaluate --
    trainer.train()
    results = trainer.evaluate()
    wandb.finish()

    return results["eval_rougeL"]

# -- 9) Run Optuna study --
pruner = SuccessiveHalvingPruner()
study = optuna.create_study(
    direction="maximize",
    pruner=pruner,
    sampler=optuna.samplers.TPESampler(seed=42)
)
study.optimize(objective, n_trials=50)

print("Best trial:")
print(study.best_trial)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Resolving data files:   0%|          | 0/31 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/31 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[I 2025-05-17 08:54:30,023] A new study created in memory with name: no-name-76498728-bf5a-4c51-952f-1d81b0e27549


...

eval/loss,▁
eval/rougeL,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇████
train/grad_norm,▂▂▃▂▂▂▃▁▅▂▅▂▆▁▄▅▄▄▅▃▃▇█▃▂▃▅▅█▂█▆▆
train/learning_rate,███▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁
train/loss,█▆▇▆▅▅▅▃▄▃▄▂▃▁▃▃▂▃▂▁▂▂▂▂▂▁▂▂▂▃▂▃▁
eval/loss,0.68933


/tmp/ipykernel_87947/295073416.py:75: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr       = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
/tmp/ipykernel_87947/295073416.py:78: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  weight_decay = trial.suggest_uniform("weight_decay", 0.0, 0.1)
/tmp/ipykernel_87947/295073416.py:81: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  lora_dropout= trial.suggest_uniform("lora_dropout", 0.0, 0.3)
/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:550: UserWarning: Model with `tie_word_embe

Step,Training Loss,Validation Loss


[I 2025-05-17 13:12:26,127] Trial 49 pruned. Trial was pruned at step 331


Best trial:
FrozenTrial(number=31, state=TrialState.COMPLETE, values=[0.7290111782758841], datetime_start=datetime.datetime(2025, 5, 17, 11, 38, 29, 967734), datetime_complete=datetime.datetime(2025, 5, 17, 11, 44, 26, 369689), params={'learning_rate': 9.596461243144801e-05, 'per_device_train_batch_size': 2, 'gradient_accumulation_steps': 1, 'weight_decay': 0.07115933746212984, 'lora_r': 16, 'lora_alpha': 32, 'lora_dropout': 0.054778160858588026}, user_attrs={}, system_attrs={'completed_rung_0': 0.7290111782758841, 'completed_rung_1': 0.7290111782758841, 'completed_rung_2': 0.7290111782758841, 'completed_rung_3': 0.7290111782758841}, intermediate_values={662: 0.7290111782758841}, distributions={'learning_rate': FloatDistribution(high=0.0001, log=True, low=1e-06, step=None), 'per_device_train_batch_size': CategoricalDistribution(choices=(2, 4)), 'gradient_accumulation_steps': CategoricalDistribution(choices=(1, 2, 4)), 'weight_decay': FloatDistribution(high=0.1, log=False, low=0.0, step

In [3]:
study.best_params

{'learning_rate': 9.596461243144801e-05,
 'per_device_train_batch_size': 2,
 'gradient_accumulation_steps': 1,
 'weight_decay': 0.07115933746212984,
 'lora_r': 16,
 'lora_alpha': 32,
 'lora_dropout': 0.054778160858588026}

In [11]:
9.596461243144801e-05/4

2.3991153107862002e-05